In [ ]:
#import packages and authenticate my Twitter API
import sys
!{sys.executable} -m pip install tweepy
!python3 -m textblob.download_corpora
import os
import tweepy as tw
import pandas as pd
from textblob import TextBlob
import itertools
import collections
import nltk
from nltk.corpus import stopwords
import re
import networkx
from textblob.classifiers import NaiveBayesClassifier
consumer_key= 'sFUarGpGxd2g8ffveZlP12abB'
consumer_secret= 'gPRN4tosQKk52tSZ7MG5A6V9h7ln8sTPAYpWp8L52UizYqbf4K'
access_token= '1169759758070505472-Y4iKdDZia6v1fMyXX3Qcmp1k3E10Iz'
access_token_secret= 'CWrLNVwAFXchbpzNHorvcxlli8NAA0DuprB0YrJUEJxu6'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)


You should consider upgrading via the 'pip install --upgrade pip' command.
[nltk_data] Downloading package brown to /home/nbuser/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /home/nbuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/nbuser/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nbuser/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
#main class created to process tweets
class ProcessTweets:
    def __init__(self):
        pass
        
    #methods to filter stopwords, emoticons and common emojis
    def stopwords(self):
        emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
    # Sad Emoticons
        emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
    #Emoji patterns
        emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)
        emoticons = emoticons_happy.union(emoticons_sad)
        nltk.download('stopwords')
        stop_words = set(stopwords.words('english'))
        stop_words.update(['&','*','|','-','go','find','~','/','|','0','1','2','3','4','5','6','7','8','9','10','news','me','place','fan',',','fan/','.','ig','follow','AT_USER','URL','url'])
        stop_words.update(emoticons)
        new_stop_words = set(stop_words)
        return new_stop_words
    #filter tweet and subsitute for @ symbols and URLs
    def processtweet(self, tweet):
        tweet = tweet.lower()
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet)
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet)
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
        tweet = re.sub(r':', '', tweet)
        tweet = re.sub(r'‚Ä¶', '', tweet)
        tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
        
        return tweet
    #basic NLP training for training set
    def basictraining(self):
        import csv
        global cl
        with open('basictraining.csv', 'r') as basic:
            cl = NaiveBayesClassifier(basic, format="csv")
        with open('ebonics_training.csv', newline='') as f:
            reader = csv.reader(f)
            data = list(reader)
            cl.update(data)
        with open('traintweets.csv', newline='') as f:
            reader = csv.reader(f)
            data = list(reader)
            cl.update(data)
            
        
    #analyze training data set and retrain it as well    
    def analyzeoldtweets(self,search_words,date_since,date_until,resulttype,count,geo):
        sentiments = []
        import csv
        global cl
        with open('basictraining.csv', 'r') as basic:
            cl = NaiveBayesClassifier(basic, format="csv")
        with open('ebonics_training.csv', newline='') as f:
            reader = csv.reader(f)
            data = list(reader)
            cl.update(data)
        with open('traintweets.csv', newline='') as f:
            reader = csv.reader(f)
            data = list(reader)
            cl.update(data)
        tweets = tw.Cursor(api.search,
              q=search_words,result_type = resulttype,
              lang="en",
              since=date_since,until=date_until,geocode=geo).items(count)
        for t in tweets:    
            if (not t.retweeted) and ('RT @' not in t.text):
                tweettext = self.processtweet(t.text)
                tweet_sentiment = cl.classify(t.text)
                sentiments.append({'User':t.user.screen_name,'Tweet': tweettext,'Sentiment':tweet_sentiment,'Bios':t.user.description,'Time':t.created_at,'Retweets':t.retweet_count,'Likes':t.favorite_count,'Followers':t.user.followers_count})
        sent = pd.DataFrame(sentiments)
        return sent
    
    #analyze test dataset and provide sentiments
    def analyzenewtweets(self,search_words,resulttype,count):
        sentiments=[]
        tweets = tw.Cursor(api.search,
              q=search_words,result_type = resulttype,
              lang="en").items(count)
        for t in tweets:    
            if (not t.retweeted) and ('RT @' not in t.text):
                tweettext = self.processtweet(t.text)
                tweet_sentiment = cl.classify(t.text)
                sentiments.append({'User':t.user.screen_name,'Tweet': tweettext,'Sentiment':tweet_sentiment,'Bios':t.user.description,'Time':t.created_at,'Retweets':t.retweet_count,'Likes':t.favorite_count,'Followers':t.user.followers_count})
        sent = pd.DataFrame(sentiments)
        return sent
    
    #find frequency of most common terms in Bio
    def bioprocessor(self, output,stopwords):
        bios = output['Bios'].tolist()
        bios = list(itertools.chain(bios))
        words = [tweet.lower().split() for tweet in bios]
        clean_tweets = [[word for word in tweet_words if not word in stopwords] for tweet_words in words]
        all_clean_words = list(itertools.chain(*clean_tweets))
        newcount = collections.Counter(all_clean_words)
        freq = pd.DataFrame(newcount.most_common(30),
                             columns=['words', 'count'])
        
        return freq
    #find frequency of most common terms in Tweets
    def tweetprocessor(self,output,stopwords):
        body = output['Tweet'].tolist()
        body = list(itertools.chain(body))
        tweetwords = [tweet.lower().split() for tweet in body]
        clean_tweetwords = [[word for word in tweet_words if not word in stopwords] for tweet_words in tweetwords]
        all_clean_tweetwords = list(itertools.chain(*clean_tweetwords))
        tweetcount = collections.Counter(all_clean_tweetwords)
        tweetfreq = pd.DataFrame(tweetcount.most_common(30),
                         columns=['words', 'count'])
        return tweetfreq

In [3]:
processor = ProcessTweets()
search_words = "#DarkLaneDemoTapes"
day1 = "2020-04-30"
day2 = "2020-05-01"
day3 = "2020-05-02"
day4 = "2020-05-03"
day5 = "2020-04-08"
day6 = "2020-04-09"
day7 = "2020-04-10"
result_type = 'recent'
popular = 'popular'
geo = "40.714353,-74.00597299999998,20km"
count = 1500

# Create first batch of untrained tweets and train them

In [ ]:
untrainedtweets = processor.analyzeoldtweets(search_words,day1,day3,result_type,count,geo)
untrained.to_csv('untrainedtweets.csv')
#correct sentiment then copy and paste to trainedtweets.csv

# Reprocess Tweets and then first see first batch of sentiment tweets

In [ ]:
firstbatch = processor.analyzeoldtweets(search_words,day1,day3,result_type,count,geo)
firstbatch_sent = firstbatch['Sentiment'].value_counts(normalize=True)*100
firstbatch.to_csv('darklanetweets.csv', header=False, mode = 'a')
pd.read_csv('darklanetweets.csv') 

# Pull latest Tweets and retrain them, add them to 'trainedtweets'

In [ ]:
newprocessor = ProcessTweets()
newtweets = newprocessor.analyzenewtweets(search_words,result_type,count)
newtweets.to_csv('untrainedtweets.csv')
#correct sentiment then copy and paste to trainedtweets.csv

# Process Latest Tweets and add them to overall csv

In [ ]:
newtweets = newprocessor.analyzenewtweets(search_words,result_type,count)
newtweets.to_csv('darklantetweets.csv',header=False,mode='a')
pd.read_csv('darklanetweets.csv')

# Analyze Sentiment Overall So Far

In [ ]:
alltweets = pd.read_csv('darklanetweets.csv')
sent_count = alltweets['Sentiment'].value_counts(normalize=True)*100
sent_count

# Bio Freq & Tweet Freq

In [ ]:
new_stop_words = processor.stopwords()
biofreq = processor.bioprocessor(alltweets,new_stop_words)
biofreq

In [ ]:
tweetfreq = processor.tweetprocessor(alltweets,new_stop_words)
tweetfreq

In [6]:
day1tweets = processor.analyzeoldtweets(search_words,day1,day2,result_type,count,geo)
day1tweets
day1_sent_count = day1tweets['Sentiment'].value_counts(normalize = True)*100
day1_sent_count

pos    100.0
Name: Sentiment, dtype: float64

In [8]:
day2tweets = processor.analyzeoldtweets(search_words,day1,day3,result_type,count,geo)
day2tweets.to_csv('darklanetweets.csv')
day2tweets
day2_sent_count = day2tweets['Sentiment'].value_counts(normalize = True)*100
day2_sent_count

pos        94.886364
neg         3.977273
neutral     1.136364
Name: Sentiment, dtype: float64

In [4]:
day3tweets = processor.analyzeoldtweets(search_words,day3,day4,result_type,count,geo)
day3tweets.to_csv('darklanetweets.csv', header=False, mode = 'a')
day3_sent_count = day3tweets['Sentiment'].value_counts(normalize = True)*100
day3_sent_count

pos    93.75
neg     6.25
Name: Sentiment, dtype: float64

In [7]:
newprocessor = ProcessTweets()
newtweets = newprocessor.analyzenewtweets(search_words,result_type,count)
newtweets.to_csv('darklanetweets.csv',header=False,mode='a')

In [8]:
alltweets = pd.read_csv('darklanetweets.csv')
alltweets_count = alltweets['Sentiment'].value_counts(normalize=True)*100
alltweets_count

pos        97.861635
neg         1.886792
neutral     0.251572
Name: Sentiment, dtype: float64

In [9]:
alltweets

,Unnamed: 0,Bios,Followers,Likes,Retweets,Sentiment,Time,Tweet,User
0,0,NYC🗽| NJ 🧱 Last account suspended @ 4K 🙄 Model...,631,2,0,pos,2020-05-01 23:57:33,okay so i know drake never disappoints but whe...,therealemiliaa
1,1,FL📍 | DMV | NYC Jeremiah 29:11 The sparkle dep...,1280,1,0,neg,2020-05-01 23:53:53,me watching everyone praise drake for the new ...,Kimeraaa_
2,2,made in 🇵🇷 : unOrdinary : ♥️⚓️✝️ : a mi na...,7,0,0,pos,2020-05-01 23:39:14,AT_USER who are you talking about in losses? f...,Rebulera89
3,3,Social Media Strategist @Tasty✨,418,12,4,pos,2020-05-01 23:11:23,i made this for you AT_USER darklanedemotapes URL,angiekraz
4,4,writing a book on the link between brain damag...,969,0,0,pos,2020-05-01 23:09:02,ight go ahead what s the best song on darklane...,detectiv3konan
5,5,Making Culture Pop.,2215798,189,23,pos,2020-05-01 22:32:56,from the hills of los angeles to new york ci...,Complex
6,6,Father God forgive me ima be sinning for a min...,6,0,0,pos,2020-05-01 22:21:31,who had the better quarantine bars AT_USER or...,ike3hunna
7,7,III Nana 💙,49,0,0,pos,2020-05-01 22:20:19,i literally love every song on AT_USER album d...,vgotthetea1
8,8,🅙🅤🅢🅣 🅐 🅢🅐🅜🅟🅛🅔 ➋ Λｖａｉｌａｂｌｅ ＯＮ 𝙰𝙻𝙻 𝙼𝚊𝚓𝚘𝚛 MUSIC P...,21488,0,0,pos,2020-05-01 22:13:29,thank god 4 darklanedemotapes,ClassyBlassy
9,9,"If I see it, so do you. Mike Maccagnan should ...",4661,2,1,pos,2020-05-01 21:52:26,drake canada montreal jamaica morelife houston...,klew24


In [11]:
tweetfreq = processor.tweetprocessor(alltweets,new_stop_words)
tweetfreq

,words,count
0,darklanedemotapes,623
1,at_user,268
2,drake,262
3,new,90
4,like,66
5,album,49
6,song,46
7,time,41
8,love,39
9,one,34
